In [ ]:
import os
from functools import partial
from pathlib import Path
from typing import List, Dict, Optional, Union

import numpy as np
import pandas as pd
from gluonts.dataset import Dataset
from gluonts.dataset.repository.datasets import (
    get_dataset,
    dataset_names as gluonts_datasets,
)
from gluonts.time_feature.seasonality import get_seasonality
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import mase, smape

## ExperimentHandler

In [ ]:
class ExperimentHandler:
    def __init__(
        self,
        dataset: str,
        quantiles: List[float] = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        results_dir: str = "./results",
        models_dir: str = "./models",
    ):
        try:
            from gluonts.dataset import Dataset
            from gluonts.dataset.repository.datasets import (
                get_dataset,
                dataset_names as gluonts_datasets,
            )
            from gluonts.time_feature.seasonality import get_seasonality
        except ImportError:
            print('For ExperimentHandler install via `pip install -e ".[dev, experiment]"`')

        if dataset not in gluonts_datasets:
            raise Exception(
                f"dataset {dataset} not found in gluonts "
                f"available datasets: {', '.join(gluonts_datasets)}"
            )
        self.dataset = dataset
        self.quantiles = quantiles
        self.results_dir = results_dir
        self.models_dir = models_dir
        # defining datasets
        gluonts_dataset = get_dataset(self.dataset)
        self.horizon = gluonts_dataset.metadata.prediction_length
        if self.horizon is None:
            raise Exception(
                f"horizon not found for dataset {self.dataset} "
                "experiment cannot be run"
            )
        self.freq = gluonts_dataset.metadata.freq
        self.seasonality = get_seasonality(self.freq)
        self.gluonts_train_dataset = gluonts_dataset.train
        self.gluonts_test_dataset = gluonts_dataset.test
        self._create_dir_if_not_exists(self.results_dir)

    @staticmethod
    def _create_dir_if_not_exists(directory: str):
        Path(directory).mkdir(parents=True, exist_ok=True)

    @staticmethod
    def _transform_gluonts_instance_to_df(
        ts: dict,
        last_n: int | None = None,
    ) -> pd.DataFrame:
        start_period = ts["start"]
        start_ds, freq = start_period.to_timestamp(), start_period.freq
        target = ts["target"]
        ds = pd.date_range(start=start_ds, freq=freq, periods=len(target))
        if last_n is not None:
            target = target[-last_n:]
            ds = ds[-last_n:]
        ts_df = pd.DataFrame({"unique_id": ts["item_id"], "ds": ds, "y": target})
        return ts_df

    @staticmethod
    def _transform_gluonts_dataset_to_df(
        gluonts_dataset,
        last_n: int | None = None,
    ) -> pd.DataFrame:
        df = pd.concat(
            [
                ExperimentHandler._transform_gluonts_instance_to_df(ts, last_n=last_n)
                for ts in gluonts_dataset
            ]
        )
        df = df.reset_index(drop=True)
        return df

    @property
    def train_df(self) -> pd.DataFrame:
        train_df = self._transform_gluonts_dataset_to_df(self.gluonts_train_dataset)
        return train_df

    @property
    def test_df(self) -> pd.DataFrame:
        test_df = self._transform_gluonts_dataset_to_df(
            self.gluonts_test_dataset,
            last_n=self.horizon,
        )
        return test_df

## Evaluation Metrics

In [ ]:
def _divide_no_nan(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    """
    Auxiliary funtion to handle divide by 0
    """
    div = a / b
    div[div != div] = 0.0
    div[div == float("inf")] = 0.0
    return div

def _metric_protections(
    y: np.ndarray, y_hat: np.ndarray, weights: Optional[np.ndarray]
) -> None:
    if not ((weights is None) or (np.sum(weights) > 0)):
        raise Exception("Sum of `weights` cannot be 0")
    if not ((weights is None) or (weights.shape == y.shape)):
        raise Exception(
            f"Wrong weight dimension weights.shape {weights.shape}, y.shape {y.shape}"
        )

In [ ]:
def mqloss(
    y: np.ndarray,
    y_hat: np.ndarray,
    quantiles: np.ndarray,
    weights: Optional[np.ndarray] = None,
    axis: Optional[int] = None,
) -> Union[float, np.ndarray]:
    """Multi-Quantile Loss

    Calculates the Multi-Quantile loss (MQL) between `y` and `y_hat`.
    MQL calculates the average multi-quantile Loss for
    a given set of quantiles, based on the absolute
    difference between predicted quantiles and observed values.

    $$ \mathrm{MQL}(\\mathbf{y}_{\\tau},[\\mathbf{\hat{y}}^{(q_{1})}_{\\tau}, ... ,\hat{y}^{(q_{n})}_{\\tau}]) = \\frac{1}{n} \\sum_{q_{i}} \mathrm{QL}(\\mathbf{y}_{\\tau}, \\mathbf{\hat{y}}^{(q_{i})}_{\\tau}) $$

    The limit behavior of MQL allows to measure the accuracy
    of a full predictive distribution $\mathbf{\hat{F}}_{\\tau}$ with
    the continuous ranked probability score (CRPS). This can be achieved
    through a numerical integration technique, that discretizes the quantiles
    and treats the CRPS integral with a left Riemann approximation, averaging over
    uniformly distanced quantiles.

    $$ \mathrm{CRPS}(y_{\\tau}, \mathbf{\hat{F}}_{\\tau}) = \int^{1}_{0} \mathrm{QL}(y_{\\tau}, \hat{y}^{(q)}_{\\tau}) dq $$

    **Parameters:**<br>
    `y`: numpy array, Actual values.<br>
    `y_hat`: numpy array, Predicted values.<br>
    `quantiles`: numpy array. Quantiles between 0 and 1, to perform evaluation upon size (n_quantiles).<br>
    `mask`: numpy array, Specifies date stamps per serie to consider in loss.<br>

    **Returns:**<br>
    `mqloss`: numpy array, (single value).

    **References:**<br>
    [Roger Koenker and Gilbert Bassett, Jr., "Regression Quantiles".](https://www.jstor.org/stable/1913643)<br>
    [James E. Matheson and Robert L. Winkler, "Scoring Rules for Continuous Probability Distributions".](https://www.jstor.org/stable/2629907)
    """
    if weights is None:
        weights = np.ones(y.shape)
    if np.sum(quantiles > 1) > 0 or np.sum(quantiles < 0) > 0:
        raise Exception("`quantiles` need to be between 0 and 1")

    _metric_protections(y, y_hat, weights)
    n_q = len(quantiles)

    y_rep = np.expand_dims(y, axis=-1)
    error = y_hat - y_rep
    sq = np.maximum(-error, np.zeros_like(error))
    s1_q = np.maximum(error, np.zeros_like(error))
    mqloss = quantiles * sq + (1 - quantiles) * s1_q

    # Match y/weights dimensions and compute weighted average
    weights = np.repeat(np.expand_dims(weights, axis=-1), repeats=n_q, axis=-1)
    mqloss = np.average(mqloss, weights=weights, axis=axis)

    return mqloss

In [ ]:
def scaled_crps(y, y_hat, quantiles):
    """Scaled Continues Ranked Probability Score

    Calculates a scaled variation of the CRPS, as proposed by Rangapuram (2021),
    to measure the accuracy of predicted quantiles `y_hat` compared to the observation `y`.

    This metric averages percentual weighted absolute deviations as
    defined by the quantile losses.

    $$
    \mathrm{sCRPS}(\hat{F}_{\\tau}, \mathbf{y}_{\\tau}) = \\frac{2}{N} \sum_{i}
    \int^{1}_{0}
    \\frac{\mathrm{QL}(\hat{F}_{i,\\tau}, y_{i,\\tau})_{q}}{\sum_{i} | y_{i,\\tau} |} dq
    $$

    where $\hat{F}_{\\tau}$ is the an estimated multivariate distribution, and $y_{i,\\tau}$
    are its realizations.

    **Parameters:**<br>
    `y`: numpy array, Actual values of size (`n_series`, `horizon`).<br>
    `y_hat`: numpy array, Predicted quantiles of size (`n_series`, `horizon`, `n_quantiles`).<br>
    `quantiles`: numpy array,(`n_quantiles`). Quantiles to estimate from the distribution of y.<br>

    **Returns:**<br>
    `loss`: float.

    **References:**<br>
    - [Gneiting, Tilmann. (2011). \"Quantiles as optimal point forecasts\".
    International Journal of Forecasting.](https://www.sciencedirect.com/science/article/pii/S0169207010000063)<br>
    - [Spyros Makridakis, Evangelos Spiliotis, Vassilios Assimakopoulos, Zhi Chen, Anil Gaba, Ilia Tsetlin, Robert L. Winkler. (2022).
    \"The M5 uncertainty competition: Results, findings and conclusions\".
    International Journal of Forecasting.](https://www.sciencedirect.com/science/article/pii/S0169207021001722)<br>
    - [Syama Sundar Rangapuram, Lucien D Werner, Konstantinos Benidis, Pedro Mercado, Jan Gasthaus, Tim Januschowski. (2021).
    \"End-to-End Learning of Coherent Probabilistic Forecasts for Hierarchical Time Series\".
    Proceedings of the 38th International Conference on Machine Learning (ICML).](https://proceedings.mlr.press/v139/rangapuram21a.html)
    """
    eps = np.finfo(float).eps
    norm = np.sum(np.abs(y))
    loss = mqloss(y=y, y_hat=y_hat, quantiles=quantiles)
    loss = 2 * loss * np.sum(np.ones(y.shape)) / (norm + eps)
    return loss

In [ ]:
def mase(y, y_hat, y_tilde):
    """ Mean Absolute Scaled Error

    Calculates relative mean absolute errors, as proposed by Hyndman (2006),
    to measure the accuracy of predicted point forecast `y_hat` compared to the observation `y`.
    relative to the baseline model y_tilde.

    **Returns:**<br>
    `loss`: float.

    **References:**<br>
    """
    eps = np.finfo(float).eps
    diff1 = np.sum(np.abs(y-y_hat))
    diff2 = np.sum(np.abs(y-y_tilde))
    loss = diff1 / (diff2 + eps)
    return loss

In [ ]:
def smape(
    y: np.ndarray,
    y_hat: np.ndarray,
    weights: Optional[np.ndarray] = None,
    axis: Optional[int] = None,
) -> Union[float, np.ndarray]:
    """Symmetric Mean Absolute Percentage Error

    Calculates Symmetric Mean Absolute Percentage Error between
    `y` and `y_hat`. SMAPE measures the relative prediction
    accuracy of a forecasting method by calculating the relative deviation
    of the prediction and the observed value scaled by the sum of the
    absolute values for the prediction and observed value at a
    given time, then averages these devations over the length
    of the series. This allows the SMAPE to have bounds between
    0% and 200% which is desirable compared to normal MAPE that
    may be undetermined when the target is zero.

    $$ \mathrm{sMAPE}_{2}(\\mathbf{y}_{\\tau}, \\mathbf{\hat{y}}_{\\tau}) = \\frac{1}{H} \\sum^{t+H}_{\\tau=t+1} \\frac{|y_{\\tau}-\hat{y}_{\\tau}|}{|y_{\\tau}|+|\hat{y}_{\\tau}|} $$

    **Parameters:**<br>
    `y`: numpy array, Actual values.<br>
    `y_hat`: numpy array, Predicted values.<br>
    `mask`: numpy array, Specifies date stamps per serie to consider in loss.<br>

    **Returns:**<br>
    `smape`: numpy array, (single value).

    **References:**<br>
    [Makridakis S., "Accuracy measures: theoretical and practical concerns".](https://www.sciencedirect.com/science/article/pii/0169207093900793)
    """
    _metric_protections(y, y_hat, weights)

    delta_y = np.abs(y - y_hat)
    scale = np.abs(y) + np.abs(y_hat)
    smape = _divide_no_nan(delta_y, scale)
    smape = 2 * np.average(smape, weights=weights, axis=axis)

    if isinstance(smape, float):
        assert smape <= 2, "SMAPE should be lower than 200"
    else:
        assert all(smape <= 2), "SMAPE should be lower than 200"

    return smape

## StatsForecast baselines

In [ ]:
from scipy.stats import norm
from statsforecast import StatsForecast
from statsforecast.models import (
    AutoARIMA,
    AutoETS,
    AutoCES,
    DynamicOptimizedTheta,
    SeasonalNaive,
    ZeroModel,
)

def ensemble_forecasts(
    fcsts_df: pd.DataFrame,
    quantiles: List[float],
    model_names: List[str],
    model_name: str,
) -> pd.DataFrame:
    fcsts_df[model_name] = fcsts_df[model_names].mean(axis=1).values  # type: ignore
    # compute quantiles based on the mean of the forecasts
    sigma_models = []
    for model in model_names:
        fcsts_df[f"sigma_{model}"] = fcsts_df[f"{model}-hi-68.27"] - fcsts_df[model]
        sigma_models.append(f"sigma_{model}")
    fcsts_df[f"std_{model_name}"] = (
        fcsts_df[sigma_models].pow(2).sum(axis=1).div(len(sigma_models) ** 2).pow(0.5)
    )
    z = norm.ppf(quantiles)
    q_cols = []
    for q, zq in zip(quantiles, z):
        q_col = f"{model_name}-q-{q}"
        fcsts_df[q_col] = fcsts_df[model_name] + zq * fcsts_df[f"std_{model_name}"]
        q_cols.append(q_col)
    fcsts_df = fcsts_df[["unique_id", "ds"] + [model_name] + q_cols + ["SeasonalNaive"]]
    return fcsts_df

In [ ]:
dataset = "m1_yearly"

exp = ExperimentHandler(dataset)
train_df = exp.train_df
test_df = exp.test_df
seasonality = exp.seasonality
freq = exp.freq
horizon = exp.horizon
quantiles = exp.quantiles

print(quantiles)

# def run_statistical_baselines(dataset):
# ---------------------------- StatsForecast Code ----------------------------
series_per_core = 15
n_series = train_df["unique_id"].nunique()
n_jobs = min(n_series // series_per_core, os.cpu_count())

models = [
    AutoARIMA(season_length=seasonality),
    AutoETS(season_length=seasonality),
    AutoCES(season_length=seasonality),
    DynamicOptimizedTheta(season_length=seasonality),
    SeasonalNaive(season_length=seasonality)
]
sf = StatsForecast(
    models=models,
    fallback_model=ZeroModel(),
    freq=freq,
    n_jobs=n_jobs,
)
fcsts_df = sf.forecast(df=train_df, h=horizon, level=[68.27])

# ----------------------- Wrangle StatsForecast preds ------------------------
models = {'SCUM': ['CES', 'DynamicOptimizedTheta', 'AutoARIMA', 'AutoETS'],
          'CES': ['CES'],
          'DynamicOptimizedTheta': ['DynamicOptimizedTheta'],
          'AutoARIMA': ['AutoARIMA'],
          'AutoETS': ['AutoETS']}

for model in models:
    q_cols = [f"{model}-q-{q}" for q in quantiles]
    median_col = f"{model}-q-0.5"

    # Get Quantile Forecasts
    preds_df = ensemble_forecasts(
        fcsts_df=fcsts_df,
        quantiles=quantiles,
        model_names=models[model],
        model_name=model)

    # Reshape datasets into DeepTSv3 format
    y_df = train_df.groupby('unique_id', sort=False, )
    target_df = test_df.groupby('unique_id', sort=False, )
    preds_df = preds_df.groupby('unique_id', sort=False, )
  
    Y_hat_df = pd.DataFrame({
        "y": y_df.apply(lambda g: g["y"].to_numpy().tolist(), include_groups=False),
        "ds": y_df.apply(lambda g: g["ds"].to_numpy().tolist(), include_groups=False),
        "preds": preds_df.apply(lambda g: g[q_cols].to_numpy().tolist(), include_groups=False),
        "target": target_df.apply(lambda g: g["y"].to_numpy().tolist(), include_groups=False),
        "median_preds": preds_df.apply(lambda g: g[median_col].to_numpy().tolist(), include_groups=False),
        "snaive_preds": preds_df.apply(lambda g: g["SeasonalNaive"].to_numpy().tolist(), include_groups=False),
        }).reset_index()

    Y_hat_df.to_parquet(f"{dataset}-{model}.parquet")

for model in models:
    if os.path.exists(f"{dataset}-{model}.parquet"):
        os.remove(f"{dataset}-{model}.parquet")

## StatsForecast Evaluation

In [ ]:
quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

datasets = [
    "m1_monthly",
    "m1_quarterly",
    "m1_yearly",
    "m3_other",
    "m3_monthly",
    "m3_quarterly",
    "m3_yearly",
    "m4_hourly",
    "m4_daily",
    'm4_weekly',
    "m4_monthly",
    "m4_quarterly",
    "m4_yearly",
    "tourism_monthly",
    "tourism_quarterly",
    "tourism_yearly",
]

models = {'SCUM': ['CES', 'DynamicOptimizedTheta', 'AutoARIMA', 'AutoETS'],
          'CES': ['CES'],
          'DynamicOptimizedTheta': ['DynamicOptimizedTheta'],
          'AutoARIMA': ['AutoARIMA'],
          'AutoETS': ['AutoETS']}

mase_dfs = []
scrps_dfs = []
smape_dfs = []
for dataset in datasets:
    mase_df = {'dataset': dataset.split("_", 1)[0], 'freq': dataset.split("_", 1)[1]}
    scrps_df = {'dataset': dataset.split("_", 1)[0], 'freq': dataset.split("_", 1)[1]}
    smape_df = {'dataset': dataset.split("_", 1)[0], 'freq': dataset.split("_", 1)[1]}
    for model in models:
        Y_hat_df = pd.read_parquet(f"neurips_results/{dataset}-{model}.parquet")

        target = np.vstack(Y_hat_df['target'])
        preds = np.vstack(Y_hat_df['preds']).flatten()
        preds = np.concatenate(preds).reshape(target.shape[0], target.shape[1], -1)
        _scrps = scaled_crps(y=target, y_hat=preds, quantiles=np.array(quantiles))

        median_preds = np.vstack(Y_hat_df['median_preds']).flatten()
        median_preds = median_preds.reshape(target.shape[0], target.shape[1])
        snaive_preds = np.vstack(Y_hat_df['snaive_preds']).flatten()
        snaive_preds = snaive_preds.reshape(target.shape[0], target.shape[1])
        _mase = mase(y=target, y_hat=median_preds, y_tilde=snaive_preds)
        _smape = smape(y=target, y_hat=median_preds)

        mase_df[model] = _mase
        scrps_df[model] = _scrps
        smape_df[model] = _smape

    mase_dfs.append(mase_df)
    scrps_dfs.append(scrps_df)
    smape_dfs.append(smape_df)

mase_df = pd.DataFrame(mase_dfs)
scrps_df = pd.DataFrame(scrps_dfs)
smape_df = pd.DataFrame(smape_dfs)

Y_hat_df

In [ ]:
scrps_df["NBEATS"]   = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
scrps_df["NHITS"]    = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
scrps_df["MQCNN"]    = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
scrps_df["PatchTST"] = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
scrps_df["TFT"]      = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
scrps_df["Chronos"]  = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
columns = ["dataset", "freq", "NBEATS", "NHITS", "MQCNN", "PatchTST", "TFT", "Chronos", "SCUM", "CES", "DynamicOptimizedTheta", "AutoARIMA", "AutoETS"]
print(scrps_df[columns].to_latex(index=False,
                        formatters={"name": str.upper},
                        float_format="{:.4f}".format,
))
print("sCRPS")
scrps_df[columns]

In [ ]:
mase_df["NBEATS"]   = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
mase_df["NHITS"]    = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
mase_df["MQCNN"]    = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
mase_df["PatchTST"] = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
mase_df["TFT"]      = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
mase_df["Chronos"]  = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
columns = ["dataset", "freq", "NBEATS", "NHITS", "MQCNN", "PatchTST", "TFT", "Chronos", "SCUM", "CES", "DynamicOptimizedTheta", "AutoARIMA", "AutoETS"]
print(mase_df[columns].to_latex(index=False,
                       formatters={"name": str.upper},
                       float_format="{:.4f}".format,
))
print("MASE")
mase_df[columns]

In [ ]:
smape_df["NBEATS"]   = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
smape_df["NHITS"]    = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
smape_df["MQCNN"]    = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
smape_df["PatchTST"] = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
smape_df["TFT"]      = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
smape_df["Chronos"]  = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
columns = ["dataset", "freq", "NBEATS", "NHITS", "MQCNN", "PatchTST", "TFT", "Chronos", "SCUM", "CES", "DynamicOptimizedTheta", "AutoARIMA", "AutoETS"]
print(smape_df[columns].to_latex(index=False,
                       formatters={"name": str.upper},
                       float_format="{:.4f}".format,
))
print("sMAPE")
smape_df[columns]